In [1]:
#coding=utf-8
# coding=gbk
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score
from torchvision import transforms, datasets, utils
import torch.nn.functional as F

In [2]:
# 加载数据集
data_transform = {
    "train": transforms.Compose([transforms.RandomResizedCrop(224),#将给定图像随机裁剪为不同的大小和宽高比，然后缩放所裁剪得到的图像为制定的大小
    #随机裁剪后的大小: (224, 224)
                                 transforms.RandomHorizontalFlip(),#随机水平（左右）翻转，默认值为0.5
                                 transforms.ToTensor(),#图片转换成形状为(C, H, W)的Tensor格式
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
                                 #使用transforms.Normalize(mean, std)对图像按通道进行标准化，即减去均值，再除以方差。
                                 #这样做可以加快模型的收敛速度。其中参数mean和std分别表示图像每个通道的均值和方差序列。
    "val": transforms.Compose([transforms.Resize((224, 224)),  # cannot 224, must (224, 224)
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])}

image_path = 'split-garbage-dataset/'
#1、训练集（train set）：用于训练模型以及确定参数。相当于老师教学生知识的过程。

#2、验证集（validation set）：用于确定网络结构以及调整模型的超参数。相当于月考等小测验，用于学生对学习的查漏补缺。

#3、测试集（test set）：用于检验模型的泛化能力。相当于大考，上战场一样，真正的去检验学生的学习效果。

assert os.path.exists(image_path), "{} path does not exist.".format(image_path)
train_dataset = datasets.ImageFolder(root=os.path.join(image_path, "train"),
                                     transform=data_transform["train"])

test_dataset = datasets.ImageFolder(root=os.path.join(image_path, "val"),
                                        transform=data_transform["val"])

In [3]:
# 设置超参数
batch_size = 16
learning_rate = 0.002
num_epochs = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 创建数据加载器
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
class SE_Block(nn.Module):
    def __init__(self, ch_in, reduction=16):
        super(SE_Block, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)  # 全局自适应池化
        self.fc = nn.Sequential(
            nn.Linear(ch_in, ch_in // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(ch_in // reduction, ch_in, bias=False),
            nn.Sigmoid()
        )
 
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c) # squeeze操作
        y = self.fc(y).view(b, c, 1, 1) # FC获取通道注意力权重，是具有全局信息的
        return x * y.expand_as(x) # 注意力作用每一个通道上
expansion = 2

class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, groups=in_channels)
        self.pointwise_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.relu = nn.ReLU(inplace=True) 
       
    def forward(self, x):
        x = self.depthwise_conv(x)
        x = self.pointwise_conv(x)
        x = self.relu(x)
        return x

In [5]:
class MobileNetV1(nn.Module):
    def __init__(self, num_classes=1000):
        super(MobileNetV1, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.dw_separable_conv1 = DepthwiseSeparableConv(32, 64)
        self.SE1 = SE_Block(64)
        self.dw_separable_conv2 = DepthwiseSeparableConv(64, 128)
        self.SE2 = SE_Block(128)
        self.dw_separable_conv3 = DepthwiseSeparableConv(128, 128)
        self.SE3 = SE_Block(128)
        self.dw_separable_conv4 = DepthwiseSeparableConv(128, 256)
        self.SE4 = SE_Block(256)
        self.dw_separable_conv5 = DepthwiseSeparableConv(256, 256)
        self.SE5 = SE_Block(256)
        self.dw_separable_conv6 = DepthwiseSeparableConv(256, 512)
        self.SE6 = SE_Block(512)
        self.dw_separable_conv7 = DepthwiseSeparableConv(512, 512)
        self.SE7 = SE_Block(512)
        self.dw_separable_conv8 = DepthwiseSeparableConv(512, 512)
        self.SE8 = SE_Block(512)
        self.dw_separable_conv9 = DepthwiseSeparableConv(512, 512)
        self.SE9 = SE_Block(512)
        self.dw_separable_conv10 = DepthwiseSeparableConv(512, 512)
        self.SE10 = SE_Block(512)
        self.dw_separable_conv11 = DepthwiseSeparableConv(512, 512)
        self.SE11 = SE_Block(512)
        self.dw_separable_conv12 = DepthwiseSeparableConv(512, 1024)
        self.SE12 = SE_Block(1024)
        self.dw_separable_conv13 = DepthwiseSeparableConv(1024, 1024)
        self.SE13 = SE_Block(1024)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, num_classes)
 
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.dw_separable_conv1(x)
        x = self.SE1(x)
        x = self.dw_separable_conv2(x)
        x = self.SE2(x)
        x = self.dw_separable_conv3(x)
        x = self.SE3(x)
        x = self.dw_separable_conv4(x)
        x = self.SE4(x)
        x = self.dw_separable_conv5(x)
        x = self.SE5(x)
        x = self.dw_separable_conv6(x)
        x = self.SE6(x)
        x = self.dw_separable_conv7(x)
        x = self.SE7(x)
        x = self.dw_separable_conv8(x)
        x = self.SE8(x)
        x = self.dw_separable_conv9(x)
        x = self.SE9(x)
        x = self.dw_separable_conv10(x)
        x = self.SE10(x)
        x = self.dw_separable_conv11(x)
        x = self.SE11(x)
        x = self.dw_separable_conv12(x)
        x = self.SE12(x)
        x = self.dw_separable_conv13(x)
        x = self.SE13(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x
    

In [6]:
model = MobileNetV1(num_classes=6).to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 测试模型
def validate(model, device, test_loader):
    model.eval()
    loss = 0
    predicted_labels = []
    true_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss += criterion(outputs, labels).item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            print(predicted)
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    loss /= len(test_loader.dataset)
    accuracy = 100 * sum([int(predicted_labels[i] == true_labels[i]) for i in range(len(predicted_labels))]) / len(predicted_labels)
    recall = recall_score(true_labels, predicted_labels, average='macro')
    precision = precision_score(true_labels, predicted_labels, average='macro')
    f1 = f1_score(true_labels, predicted_labels, average='macro')
    kappa = cohen_kappa_score(true_labels, predicted_labels)
    return accuracy, precision, recall, f1, kappa, loss


In [ ]:
model = MobileNetV1(num_classes=6).to(device)

# 训练模型
best_model_epoch = 0
best_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        for name, param in model.named_parameters():
            if param.grad is not None:
                print(f'Layer: {name}, Gradient norm: {param.grad.norm().item()}')

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 50 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

        train_loss += loss.item() * labels.size(0)

    train_loss /= len(train_dataset)

    # 每个epoch结束后，在测试集上评估一下模型的性能
    val_acc, val_precision, val_recall, val_f1, val_kappa, val_loss = validate(model, device, test_loader)

    print('Epoch [{}/{}], Train Loss: {:.4f}, Val Loss: {:.4f}'
          .format(epoch+1, num_epochs, train_loss, val_loss))
    print('Val Accuracy: {:.2f}%'.format(val_acc))
    print('Val Recall: {:.4f}'.format(val_recall))
    print('Val Precision: {:.4f}'.format(val_precision))
    print('Val F1-score: {:.4f}'.format(val_f1))
    print('Val Kappa: {:.4f}'.format(val_kappa))
    print('-' * 25)
    
    
dummy_input = torch.rand(1, 3, 224, 224).to(device)

Layer: conv1.weight, Gradient norm: 2.7448591902005937e-13
Layer: conv1.bias, Gradient norm: 1.9143534142878477e-13
Layer: dw_separable_conv1.depthwise_conv.weight, Gradient norm: 2.37413143614959e-13
Layer: dw_separable_conv1.depthwise_conv.bias, Gradient norm: 4.798198513858432e-13
Layer: dw_separable_conv1.pointwise_conv.weight, Gradient norm: 9.824247535275554e-13
Layer: dw_separable_conv1.pointwise_conv.bias, Gradient norm: 8.458952939147113e-13
Layer: SE1.fc.0.weight, Gradient norm: 7.798768717674851e-15
Layer: SE1.fc.2.weight, Gradient norm: 3.668414567747176e-15
Layer: dw_separable_conv2.depthwise_conv.weight, Gradient norm: 7.478974037300468e-13
Layer: dw_separable_conv2.depthwise_conv.bias, Gradient norm: 4.1798240216217586e-12
Layer: dw_separable_conv2.pointwise_conv.weight, Gradient norm: 9.455840624394973e-12
Layer: dw_separable_conv2.pointwise_conv.bias, Gradient norm: 6.563423415872904e-12
Layer: SE2.fc.0.weight, Gradient norm: 2.02439939269404e-13
Layer: SE2.fc.2.weight

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Layer: conv1.weight, Gradient norm: 1.1377942858714629e-11
Layer: conv1.bias, Gradient norm: 5.338374707564153e-12
Layer: dw_separable_conv1.depthwise_conv.weight, Gradient norm: 5.490786211120868e-12
Layer: dw_separable_conv1.depthwise_conv.bias, Gradient norm: 1.1686872823657435e-11
Layer: dw_separable_conv1.pointwise_conv.weight, Gradient norm: 2.5139673329177548e-11
Layer: dw_separable_conv1.pointwise_conv.bias, Gradient norm: 2.115089700605033e-11
Layer: SE1.fc.0.weight, Gradient norm: 1.2213087529147626e-13
Layer: SE1.fc.2.weight, Gradient norm: 9.14487195279208e-14
Layer: dw_separable_conv2.depthwise_conv.weight, Gradient norm: 1.7979258706835033e-11
Layer: dw_separable_conv2.depthwise_conv.bias, Gradient norm: 9.422607832876295e-11
Layer: dw_separable_conv2.pointwise_conv.weight, Gradient norm: 2.2869825477833672e-10
Layer: dw_separable_conv2.pointwise_conv.bias, Gradient norm: 1.5871745584483676e-10
Layer: SE2.fc.0.weight, Gradient norm: 3.857524976530469e-12
Layer: SE2.fc.2.w

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Layer: conv1.weight, Gradient norm: 2.1744691117153714e-11
Layer: conv1.bias, Gradient norm: 1.0580152205730275e-11
Layer: dw_separable_conv1.depthwise_conv.weight, Gradient norm: 1.0343915828048278e-11
Layer: dw_separable_conv1.depthwise_conv.bias, Gradient norm: 2.2966647680866537e-11
Layer: dw_separable_conv1.pointwise_conv.weight, Gradient norm: 4.914969820024595e-11
Layer: dw_separable_conv1.pointwise_conv.bias, Gradient norm: 4.1423336982759906e-11
Layer: SE1.fc.0.weight, Gradient norm: 2.6794048208454713e-13
Layer: SE1.fc.2.weight, Gradient norm: 1.771659533487957e-13
Layer: dw_separable_conv2.depthwise_conv.weight, Gradient norm: 3.500903733577587e-11
Layer: dw_separable_conv2.depthwise_conv.bias, Gradient norm: 1.8360274411932664e-10
Layer: dw_separable_conv2.pointwise_conv.weight, Gradient norm: 4.472321679660496e-10
Layer: dw_separable_conv2.pointwise_conv.bias, Gradient norm: 3.1041214043625587e-10
Layer: SE2.fc.0.weight, Gradient norm: 7.496957915575919e-12
Layer: SE2.fc.2